# This notebook is to plot all my credit card records.
    # The purpose is to keep track of my expenses and to know where all my money went...

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
#---- matplotlib, my old favorite...
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='ticks')
%matplotlib inline
#---- for Bokeh, a ploter
import bokeh.plotting as bp
from bokeh.layouts import gridplot, column
from bokeh import palettes, charts
from bokeh.models import LinearAxis, DataRange1d, HoverTool, BoxSelectTool
from bokeh.charts import Histogram, Bar
TOOLS = ["pan,wheel_zoom,box_zoom,reset,crosshair, hover, box_select"]
WIDTH = 990
HEIGHT = 280
bp.output_notebook()
#---- for color, style print in iPython
from IPython.display import HTML, display
import datetime
import os
#---- for colorful print
from colorama import init
init(autoreset=True)
from colorama import Fore, Back, Style
#---- for statics
from scipy import stats

class ExpenseManager:
    def __init__(self):
        self.mitsuiVisaRecord = '../[SensitiveData/Expense/Mitsui/'
        self.JCBRecord = '../[SensitiveData/Expense/JCB/'
        print("Initializing ExpenseManager instance")

    # read one csv file, Mitsui VISA format
    def readMitsuiVisaCSV(self, filePath):
        data = pd.read_csv(filePath, encoding='shift-JIS', header=None, usecols=[0,1,2], names=('Date', 'Category', 'Expense'))
        data = data[data['Date'] != '陳　飛宇　様']
        data = data[data['Date'].notnull()]
        # convert date, expense
        data['Date'] = pd.to_datetime(data['Date'])
        data['Expense'] = pd.to_numeric(data['Expense'])
        return data
    # read one csv file, JCB format
    def readJCB(self, filePath):
        data = pd.read_csv(filePath, encoding='shift-JIS', header=None, skiprows=[0,1,2,3,4,5],usecols=[2,3,4], names=('Date', 'Category', 'Expense'))
        data = data[data['Date'].notnull()]
        # convert date, expense
        data['Date'] = pd.to_datetime(data['Date'])
        #- replace ',' with ''
        data['Expense'] = data['Expense'].str.replace(',', '')
        data['Expense'] = pd.to_numeric(data['Expense'])
        return data

    # read one folder, recurssively
    def readOneFolder(self, folderPath, whichRecord):
        for root, subdirs, files in os.walk(folderPath):
            #print('----------------')
            #print(Back.CYAN + Fore.RED + Style.BRIGHT + 
            #      'In Folder:'+root)
            for oneCsv in sorted(files):
                #print(Back.CYAN + oneCsv)
                filePath = folderPath+oneCsv
                if whichRecord == 'MitsuiVISA':
                    data = self.readMitsuiVisaCSV(filePath)
                elif whichRecord == 'JCB':
                    data = self.readJCB(filePath)
                try:
                    allData
                except NameError:   # allData not exist
                    allData = data
                else:   # allData exist
                    # concatenate 2 pandas dataframes
                    allData = pd.concat([allData, data])
        allData.index = range(len(allData))
        return allData
    
    # choose the data within [StartDate, EndDate], return a mask
    def chooseDateBasedonDate(self, dataframe, startDate, endDate):
        mask = (startDate < dataframe['Date']) & (dataframe['Date'] < endDate)
        return mask
    
em = ExpenseManager()

Loading BokehJS ...

Initializing ExpenseManager instance


In [2]:
mitsuiVisa = em.readOneFolder(em.mitsuiVisaRecord, 'MitsuiVISA')
JCB = em.readOneFolder(em.JCBRecord, 'JCB')

In [3]:
print(mitsuiVisa)

           Date                                  Category  Expense
0    2014-12-14                                      ローソン      529
1    2014-12-17                          サークルＫコマハ゛４チヨウメテン      551
2    2014-12-18                              ＡＭＡＺＯＮ．ＣＯ．ＪＰ     4920
3    2014-12-18                                      ローソン      113
4    2014-12-19                               サミットストア　成城店     2170
5    2014-12-20                               サミットストア　成城店     2989
6    2014-12-22                               サミットストア　成城店     1262
7    2014-12-22                          サークルＫコマハ゛４チヨウメテン      311
8    2014-12-24                               サミットストア　成城店     1841
9    2014-12-27                               サミットストア　成城店     1302
10   2014-12-28                               サミットストア　成城店      957
11   2014-12-28                               サミットストア　成城店     2392
12   2014-12-29                               サミットストア　成城店     1474
13   2014-12-30                               サミットストア　成城店     

1036 2017-08-31                                    すき家／ｉＤ      580


In [9]:
print(JCB)

          Date                             Category  Expense
0   2016-06-02                     フアミリ−マ−トゾウシキエキマエ      271
1   2016-06-03                          菜食健美　よこはま農園    35000
2   2016-06-03                     フアミリ−マ−トゾウシキエキマエ     1014
3   2016-06-04                                  ダイス      386
4   2016-06-05                         プロントコーポレーション     5820
5   2016-06-05                          スターバックスコーヒー      550
6   2016-06-05                        ＬＡ　ＣＩＴＴＡＤＥＬＬＡ     2851
7   2016-06-06                     フアミリ−マ−トゾウシキエキマエ      366
8   2016-06-10                   ＴＯＫＹＯ　ＴＯＷＥＲ　ＧＡＬＡＸＹ      594
9   2016-06-10                          東京タワーチケット売場      700
10  2016-06-11                             東京スカイツリー     2060
11  2016-06-11                             東京スカイツリー     1250
12  2016-06-11                             東京スカイツリー     1030
13  2016-06-11                             東京スカイツリー      515
14  2016-06-11                             東京スカイツリー      480
15  2016-06-12          

371 2017-08-05                              UDEMYEU     3600


In [10]:
dataVisa = mitsuiVisa.loc[em.chooseDateBasedonDate(mitsuiVisa, '2017-08', '2017-09')]
print(dataVisa)

           Date                               Category  Expense
953  2017-08-03                               ファミリーマート      100
954  2017-08-04                          セブン−イレブン・ジャパン      585
955  2017-08-05                   スタ−バックスコ−ヒ−　有楽町ビル１階店      432
956  2017-08-06                          セブン−イレブン・ジャパン      229
957  2017-08-06                                  ジョナサン     5007
958  2017-08-07                               ファミリーマート      266
959  2017-08-08                          セブン−イレブン・ジャパン      591
960  2017-08-08                               オカダヤモア―ズ      345
961  2017-08-09                          セブン−イレブン・ジャパン      583
962  2017-08-09                                   マルエツ     2413
963  2017-08-10                                 六本木ヒルズ      432
964  2017-08-10                   ＡＮＡインターネットチケットレスサービス    40680
965  2017-08-10                      トヨタレンタリース新札幌Ｗｅｂ決済    39420
966  2017-08-10                               ての字　大手町店     6230
967  2017-08-10                         

In [11]:
dataJCB = JCB.loc[em.chooseDateBasedonDate(JCB, '2017-08', '2017-09')]
print(dataJCB)

          Date              Category  Expense
369 2017-08-04  イモトのＷｉ−Ｆｉ　エクスコムグロ−バル     4320
370 2017-08-05              マルエツ〔食品〕     1734
371 2017-08-05               UDEMYEU     3600


In [12]:
print(dataVisa['Expense'].sum())
print(dataJCB['Expense'].sum())

268372
9654


In [20]:
#--- figure plot example
# 1. construct pandas dataframe
dataPlot = pd.DataFrame({
    'Cards': ['Visa', 'JCB'],
    'Month': ['2017-08', '2017-08'],
    'Expense': [dataVisa['Expense'].sum(), dataJCB['Expense'].sum()]
})
fig = Bar(dataPlot, label='Month', values='Expense', group='Cards', legend='top_right', tools=TOOLS, toolbar_location="above")
fig.legend.background_fill_alpha = 0.3
fig.legend.click_policy="hide"
bp.show(fig)
